In [1]:
import matplotlib.pyplot as mp
import numpy as np
import metpy.calc as mpy
from metpy.units import units
import xarray as xr
import datetime as dt


##### SCAM FORCINGS GENERATION FOR SAS PARAMETERS IN #####
### http://www2.mmm.ucar.edu/people/patton/documents/su_et_al.ACP.2016.pdf ###

## Time ##

bdate = 20130610

spd = 86400 # Seconds per day
sph = 3600  # Seconds per hour
mpd = 1440  # Mins per day
spm = 60    # Seconds per minute

In [2]:

###### Construct time varying data for 3d arrays (time,lat,lon) ######

## Generate boundary forced variables ##
shflx0 = vval[vname.index('shflx')]
vdesc_i = vdesc[vname.index('shflx')]

lhflx0 = vval[vname.index('lhflx')]
vdesc_i = vdesc[vname.index('lhflx')]

## SAS case study: Daylight variation of sfce fluxes (sfc fluxes to be converted to energy flux) ##
shflx = np.copy(var_3d)  
shflx[:,0,0] = shflx0*cp_air*np.sin(np.pi*(time-iop_zstart)/tperiod)

## Initialize "Dataset" ##
iop_out = xr.Dataset({'shflx': (['time','lon','lat'],  shflx)},
                     coords={'tsec': ('tsec',time),  
                             'lev': ('lev',plevs), 
                             'lon': ('lon',[iop_lon.astype(float64)]), 
                             'lat': ('lat', [iop_lat.astype(float64)]) })

## IOP out add attributes ##
iop_out.attrs['title'] = 'Southeast Atmosphere Study (SAS) campaign: Ideal day for the Mixed Layer Model (MXLCH, it can be accessed athttps://github.com/classmodel/mxlch)'
iop_out.attrs['iop_file'] = iop_file_out
iop_out.attrs['publication'] = 'https://doi.org/10.5194/acp-16-7725-2016'
iop_out.attrs['creation_date'] = str(dt.datetime.today())
iop_out.attrs['creation_date'] = 'Rich Neale, NCAR'

## Dimenstion/Coord info.
iop_out.lat.attrs = iop_in.lat.attrs
iop_out.lon.attrs = iop_in.lon.attrs

## Copy attributes of tsec from iop_in ##
iop_out.coords['tsec'] = ('time', time)
iop_out.tsec.attrs['long_name'] = iop_in.tsec.attrs['long_name'] 
iop_out.tsec.attrs['units'] = iop_in.tsec.attrs['units'] 

## LHFLX ##
lhflx = np.copy(var_3d)  
lhflx[:,0,0] = 0.001*lhflx0*Lv*np.sin(np.pi*(time-iop_zstart)/tperiod) # Convert g/kg -> kg/kg
iop_out['lhflx'] = (iop_out_c3, lhflx)

## Quick plots ##

fig,ax = mp.subplots(nrows=1, ncols=2,figsize=(12, 5))
fig.suptitle('lhflx/shflx')
ax[0].plot((time/sph)-6,lhflx[:,0,0])
ax[1].plot((time/sph)-6,shflx[:,0,0])



NameError: name 'vval' is not defined

In [ ]:
### Constructing vertical profiles ###
## Read in SCAM IOP template here (split into a function at some point)
#plevs = np.array([10, 20, 100, 150, 200, 300, 400, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 975, 1000])
#plevs = np.arange(10,1000,10)


## Convert trop theta values into temperature given plevs ##
#plevs = iop_in['lev']


ps = 101500. # psurf [pa]
p0 = 100000. # pref [pa]

## Z of plevels, tricky don't have T yet ##
dp_levs = np.diff(plevs)


##### Construct profile ICs #######

## 1. Grab CASE values ##
pblh =  vval[vname.index('pblh')]

the_ft = vval[vname.index('the_trop')]
the_bl = vval[vname.index('the_bl')]
dthedp_ft =  vval[vname.index('the_lr')]

q_ft = vval[vname.index('q_trop')]
q_bl = vval[vname.index('q_bl')]
dqdp_ft = vval[vname.index('q_lr')]



## 2. Find PBL top level ##

# Temperature profile based just on the_bl
t_bl = the_bl*(plevs/p0)**r_cp  

# Assumed t is just below 800mb
p_bl_top = 80000. # hpa estimate pf PBL top.
t_bl_ave = np.average(t_bl[np.where(plevs >= p_bl_top)])

# Z levels based on t_bl
z_plevs = (r_gas/grav)*t_bl*np.log(ps/plevs)

# Where's the PBL?
ipbl_levs = np.where(z_plevs <= pblh)
npbl_levs = np.size(ipbl_levs)
nft_levs = nplevs-npbl_levs

ipbl_min = np.amin(ipbl_levs)


## 3. Use Gradient+mean and specified theta, q for profiles ##

q_plevs = np.full(nplevs, q_bl)
the = np.full(nplevs, the_bl) # Initialize theta pbl as numpys

# Construct profiles in the FT, starting at ipbl+ and working up

for ip in range(ipbl_min-1,-1,-1):
    
    # Temp profile
    t_ip = the[ip-1]*(plevs[ip-1]/p0)**r_cp # Temp at the level below
 
    rho = plevs[ip]/(r_gas*t_ip)  # Density at lev-=ip
    dz = dp_levs[ip]/(rho*grav)
    the[ip] = the[ip+1]+dthedp_ft*dz
    
    # q profile 
 
    q_plevs[ip] = q_plevs[ip+1]+dqdp_ft*dz

     
## Set temp ##
temp_plevs = 8+the*(plevs/p0)**r_cp

## Don't let q go below a minimum (linear gradient will do that), or tail the value expoentially
#q_plevs[np.where(q_plevs <= 0)] = 0.1

########
## q Exponential tailing ##
########

qtail = 2.0  # Value of q at which to start tailing
pexp = 200. # Pressure at which to do 2/3 of the reduction (higher values squeeze closer to zero q)
dqdt_mult = 20. # Multiplier to send q to zero faster with height.

ip0 = np.where(q_plevs <= qtail)
ip0m = np.max(ip0)

## Tailing off exp.
ew = np.exp((dqdt_mult*(plevs[ip0]-plevs[ip0m])/(plevs[ip0m]-pexp))) # Exponential weights
q_plevs[ip0] = q_plevs[ip0m]*ew

print(plevs)
## RELHUM ##
plevs_mb = plevs/100.
tempk_plevs = temp_plevs-273.16 
qkg_plevs = q_plevs/1000.

esat = 6.112*np.exp((17.67*tempk_plevs)/(tempk_plevs+243.5))
eair = qkg_plevs*plevs_mb/(0.378*qkg_plevs+0.622)

rh_plevs = 100.*eair/esat

## Quick plots ##

fig1,axs = mp.subplots(nrows=1, ncols=3,figsize=(12, 5))
fig1.suptitle('T/q')
axs[0].invert_yaxis()
axs[1].invert_yaxis()
axs[2].invert_yaxis()

axs[0].plot(temp_plevs.transpose(), plevs)
axs[1].plot(q_plevs.transpose(), plevs)
axs[2].plot(rh_plevs.transpose(), plevs)
#axs[2].axvline(0, color='black',linestyle="-")



#z_plevs = (r_gas/grav)*np.mean(temp_plevs)*np.log(ps/plevs)

In [ ]:


# Constant value copying

# Omega (units are /s which don't make sense, maybe m/s)
w_sub = vval[vname.index('w_sub')]
omega = np.copy(var_4d) # Copy 4D array

# divt and divq (horizontal advection)
q_adv = vval[vname.index('q_adv')]
the_adv = vval[vname.index('the_adv')]

divT = np.copy(var_4d) # Copy 4D array, need to convert to Temp in the array u.del(the)->u.del(T)
divq = np.full_like(var_4d,0.001*q_adv) # Single value overwrite everywhere, convert g/kg/s->kg/kg/s

# Calculated above
T = np.copy(var_4d)  
q = np.copy(var_4d) 


## Send variables to "dataset" ##
for it in range(0, ntsteps):
   T[it,:,0,0] = temp_plevs.data[:]
   q[it,:,0,0] = q_plevs.data[:]
   divT[it,:,0,0] = the_adv*(plevs/p0)**r_cp # Constant in theta bbut not in T.

   rho = plevs/(r_gas*t_ip)  # Convert dw/dz -> dw/dp
   omega[it,:,0,0] = w_sub/(rho*grav)


## Write To IOP file ##

# 3D #
iop_out.lhflx.attrs = iop_in.lhflx.attrs 
iop_out.shflx.attrs = iop_in.shflx.attrs 

# 4D #
iop_out['T'] = (iop_out_c4, T)
iop_out['q'] = (iop_out_c4, np.float_(q/1000.)) # Funny SCAM requirments

if lno_lsf: # Set ls forcings to zero
    divq.fill(0.) ; divT.fill(0.) ; omega.fill(0.)
    
iop_out['divq'] = (iop_out_c4, divq)
iop_out['divT'] = (iop_out_c4, divT)
iop_out['omega'] = (iop_out_c4, omega)

for iv in iop_out.data_vars:
    iop_out[iv].attrs = iop_in[iv].attrs 
    print(iop_in[iv].attrs)
    iop_out[iv].attrs['_FillValue'] = -9999.
    iop_out[iv].attrs['missing_value'] = -9999. 

iop_out.attrs['creation_date'] = str(dt.datetime.now())
iop_out['bdate'] = bdate

#### Send everything to output IOP file
iop_out.to_netcdf(iop_file_out)

##### END #####
